In [1]:
from scipy.stats import qmc
import pandas as pd
import numpy as np
import time
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import random
import plotly.express as px
import plotly.graph_objects as go

caminho_do_projeto = r'C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao'

if caminho_do_projeto not in sys.path:
    sys.path.insert(0, caminho_do_projeto)

from projeto_aplicado.modelos.otimizador_ag import OtimizadorAG
from projeto_aplicado.modelos.otimizador_base import Otimizador
from projeto_aplicado.modelos.analisador import AnalisadorDeSolucao
from projeto_aplicado.modelos.otimizador_ag_fast import OtimizadorAGFast


# Configurações de visualização
sns.set_theme(style="whitegrid")
pd.set_option('display.max_rows', 100)

CONFIG_BASE = {
    "ARQUIVOS_DADOS": {
        "disciplinas": "disciplinas.csv",
        "professores": "docentes.csv",
        "preferencias": "preferencias.csv",
        "conflitos": "matriz_conflitos.csv" 
    }
}

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [36]:
# 1. Definir os limites dos parâmetros
limites_inferiores = [10,  0, 0.0, 2, 0]  # Pop, Cross, Mut, Torneio, Elitismo
limites_superiores = [500, 1.0, 0.5, 10, 0.5] # Limites superiores (exclusivos para inteiros se usar logica simples, mas aqui sao continuos)

# 2. Criar o amostrador Latin Hypercube
n_amostras = 150 # Quantas simulações você quer rodar?
n_parametros = 5
sampler = qmc.LatinHypercube(d=n_parametros, seed=42)
sample = sampler.random(n=n_amostras)

# 3. Escalar as amostras (que vêm entre 0 e 1) para os seus limites reais
l_bounds = np.array(limites_inferiores)
u_bounds = np.array(limites_superiores)
amostras_reais = qmc.scale(sample, l_bounds, u_bounds)

# 4. Criar DataFrame e ajustar tipos (int vs float)
df_params = pd.DataFrame(amostras_reais, columns=['n_populacao', 'taxa_crossover', 'taxa_mutacao', 'tamanho_torneio', 'elitismo_pct'])

# Ajustar inteiros
df_params['n_populacao'] = df_params['n_populacao'].astype(int)
df_params['tamanho_torneio'] = df_params['tamanho_torneio'].astype(int)
df_params['n_elitismo'] = (df_params['elitismo_pct'] * df_params['n_populacao']).astype(int)

In [45]:
dados_sumario = []
dados_historico = []

for index, row in df_params.iterrows():
    print(f"Execução {index}: {row.to_dict()}")
    CONFIG_AG = {
    **CONFIG_BASE,
    "AG_PARAMS": {
        "n_populacao": row['n_populacao'].astype(int),
        "n_geracoes": 100,
        "taxa_crossover": row['taxa_crossover'],
        "taxa_mutacao": row['taxa_mutacao'],
        "tamanho_torneio": row['tamanho_torneio'].astype(int),
        "fator_penalidade": 10000,
        "tamanho_elite": row['n_elitismo'].astype(int)
    }
    }

    otimizador = OtimizadorAGFast(config=CONFIG_AG)

    start = time.time()
    resultado = otimizador.resolver() 
    tempo = time.time() - start

    dados_sumario.append({
        "execucao": int(index + 1),
        "n_populacao": int(row['n_populacao']),
        "geracao_melhor_global": int(np.argmax([m['melhor_geracao'] for m in resultado["metricas_iteracao"]]) + 1),
        "taxa_crossover": row['taxa_crossover'],
        "taxa_mutacao": row['taxa_mutacao'],
        "tamanho_torneio": int(row['tamanho_torneio']),
        "n_elitismo": int(row['n_elitismo']),
        "melhor_fitness_final": int(resultado["valor_objetivo"]),
        "tempo_total_seg": tempo
    })

    df_hist = pd.DataFrame(resultado["metricas_iteracao"])
    df_hist["execucao"] = index +1
    dados_historico.append(df_hist)

df_resumo = pd.DataFrame(dados_sumario)
df_hist_all = pd.concat(dados_historico, ignore_index=True)

df_resumo.to_csv('resultados_analises_simulacoes/ag_estudo_sensibilidade_parametros_resumo.csv', index=False)
df_hist_all.to_csv('resultados_analises_simulacoes/ag_estudo_sensibilidade_parametros_historico.csv', index=False)

Execução 0: {'n_populacao': 494.0, 'taxa_crossover': 0.14374081040165299, 'taxa_mutacao': 0.27380467360029537, 'tamanho_torneio': 9.0, 'elitismo_pct': 0.24968607550704117, 'n_elitismo': 123.0}
Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 0 alocações foram fixadas.

--- Iniciando Processo de Otimização ---
--- Processo de Otimização Concluído ---
Execução 1: {'n_populacao': 264.0, 'taxa_crossover': 0.541592401986731, 'taxa_mutacao': 0.06737978564907682, 'tamanho_torneio': 7.0, 'elitismo_pct': 0.32849871354034815, 'n_elitismo': 86.0}
Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparad

In [13]:
dados_sumario = []
dados_historico = []
df_params = pd.DataFrame({'simulacao':[x for x in range(1, 101)]})
df_params['n_populacao'] = 200
df_params['taxa_crossover'] = 0.8
df_params['taxa_mutacao'] = 0.1
df_params['tamanho_torneio'] = 9
df_params['n_elitismo'] = 60
for index, row in df_params.iterrows():
    print(f"Execução {index}: {row.to_dict()}")
    CONFIG_AG = {
    **CONFIG_BASE,
    "AG_PARAMS": {
        "n_populacao": 200,
        "n_geracoes": 100,
        "taxa_crossover": 0.8,
        "taxa_mutacao": 0.1,
        "tamanho_torneio": 9,
        "fator_penalidade": 10000,
        "tamanho_elite": 60
    }
    }

    otimizador = OtimizadorAGFast(config=CONFIG_AG)

    start = time.time()
    resultado = otimizador.resolver() 
    tempo = time.time() - start

    dados_sumario.append({
        "execucao": int(index + 1),
        "geracao_melhor_global": int(np.argmax([m['melhor_geracao'] for m in resultado["metricas_iteracao"]]) + 1),
        "melhor_fitness_final": int(resultado["valor_objetivo"]),
        "tempo_total_seg": tempo
    })

    #df_hist = pd.DataFrame(resultado["metricas_iteracao"])
    #df_hist["execucao"] = index +1
    #dados_historico.append(df_hist)

df_resumo = pd.DataFrame(dados_sumario)
#df_hist_all = pd.concat(dados_historico, ignore_index=True)

df_resumo.to_csv('resultados_analises_simulacoes/ag_confirmacao_parametros.csv', index=False)
#df_hist_all.to_csv('resultados_analises_simulacoes/ag_dados_completos.csv', index=False)

Execução 0: {'simulacao': 1.0, 'n_populacao': 200.0, 'taxa_crossover': 0.8, 'taxa_mutacao': 0.1, 'tamanho_torneio': 9.0, 'n_elitismo': 60.0}
Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 0 alocações foram fixadas.

--- Iniciando Processo de Otimização ---
--- Processo de Otimização Concluído ---
Execução 1: {'simulacao': 2.0, 'n_populacao': 200.0, 'taxa_crossover': 0.8, 'taxa_mutacao': 0.1, 'tamanho_torneio': 9.0, 'n_elitismo': 60.0}
Buscando dados no diretório: C:\Users\kmenezes\OneDrive - unb.br\tcc_organizacao\dados
Dados brutos carregados.
0 alocações fixas validadas e separadas.
28 de 28 professores responderam ao formulário.
Dados preparados. O problema foi reduzido para a otimização.
Otimizador base inicializado. 0 alocações foram fixada

In [18]:
df_resumo['tempo_total_seg'].describe()

count    100.000000
mean       7.302292
std        2.612534
min        4.851067
25%        5.653266
50%        6.127012
75%        8.090316
max       16.849280
Name: tempo_total_seg, dtype: float64